In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules/").absolute().__str__())

from data_utils import load_dataset

In [2]:
dataset = load_dataset("../datasets/final_fq17_2/final_fq17_with_limit_10_2/final_fq17-generated_prompt-executed.parquet.gzip")
dataset.head()

,query,description,context,prompt,num_tokens,start_with_SELECT,query_templated,basic_prompt,basic_num_tokens,basic_result,...,basic_is_skipped,basic_is_prompt_too_long,templated_prompt,templated_num_tokens,templated_result,templated_full_answer,templated_is_skipped,templated_is_prompt_too_long,execution,executed_query
index,,,,,,,,,,,,,,,,,,,,,
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[INST] QUERY=""SELECT ?property ?propertyType ?...",225,"[""Write a SparQL query to retrieve Wikidata pr...",...,False,False,"[INST] QUERY=""PREFIX bd: <http://www.bigdata.c...",288,"[""Write a SparQL query to retrieve Wikidata pr...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,"[{'property': {'type': 'uri', 'value': 'http:/...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296,True,PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...,"[INST] QUERY=""SELECT ?id ?idLabel ?idDescripti...",258,"[""Write a SparQL query to retrieve Wikidata ID...",...,False,False,"[INST] QUERY=""PREFIX bd: <http://www.bigdata.c...",321,"[""Write a SparQL query to retrieve Wikidata ID...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,"[{'id': {'type': 'uri', 'value': 'http://www.w...",PREFIX bd: <http://www.bigdata.com/rdf#>\nPREF...
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197,True,PREFIX wd: <http://www.wikidata.org/entity/>\n...,"[INST] QUERY=""SELECT (COUNT(?article) AS ?coun...",158,"[""Write a query to count the number of scienti...",...,False,False,"[INST] QUERY=""PREFIX wd: <http://www.wikidata....",200,"[""Write a SparQL query to retrieve the count o...","{'content': ' 1. ""Write a SparQL query to retr...",False,False,timeout,PREFIX wd: <http://www.wikidata.org/entity/>\n...
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203,True,PREFIX wd: <http://www.wikidata.org/entity/>\n...,"[INST] QUERY=""SELECT (COUNT(DISTINCT ?article)...",164,"[""Write a query to find the number of distinct...",...,False,False,"[INST] QUERY=""PREFIX wd: <http://www.wikidata....",209,"[""Write a query to find the number of distinct...","{'content': ' 1. ""Write a query to find the nu...",False,False,[{'count': {'datatype': 'http://www.w3.org/200...,PREFIX wd: <http://www.wikidata.org/entity/>\n...
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186,True,PREFIX wdt: <http://www.wikidata.org/prop/dire...,"[INST] QUERY=""SELECT (COUNT(?item) AS ?count)\...",147,"[""Write a query to determine the number of ite...",...,False,False,"[INST] QUERY=""PREFIX wdt: <http://www.wikidata...",171,"[""Write a SparQL query to find the number of W...","{'content': ' 1. ""Write a SparQL query to find...",False,False,[{'count': {'datatype': 'http://www.w3.org/200...,PREFIX wdt: <http://www.wikidata.org/prop/dire...


In [3]:
all_duplicates = dataset.loc[dataset.duplicated(subset=['query'], keep=False)]
distinct_duplicates = all_duplicates["query"].drop_duplicates()
distinct_duplicates

index
53      SELECT DISTINCT ?paper ?paperLabel ?licenceLab...
137     SELECT ?city ?cityLabel ?image ?cityB\nWHERE {...
142     SELECT ?twit ?item ?itemLabel ?itemDescription...
149     SELECT ?from ?fromLabel ?to ?toLabel ?rgb WHER...
155     SELECT ?person ?personLabel ?positionLabel ?da...
                              ...                        
2468    SELECT ?item ?libelle ?coord ?image ?layer WHE...
2496    SELECT DISTINCT ?a ?aLabel ?b ?bLabel\n{?a wdt...
2556    SELECT ?language ?languageLabel ?lexemes ?with...
2629    SELECT ?occupation ?occupationLabel (COUNT(*) ...
2652    SELECT (count(?album) as ?albums) ?P407Label ?...
Name: query, Length: 65, dtype: object

In [65]:
duplicate_iloc = 65
duplicate_query = distinct_duplicates.iloc[duplicate_iloc]
duplicate_queries = dataset[['query', 'description', 'context', 'query_templated']].loc[dataset['query'] == duplicate_query]

print(f"=== Query")
print(duplicate_queries.iloc[0]["query_templated"])
for index, row in duplicate_queries.iterrows():
    print("-"*10 + "v"*5 + f" row index {index} " + "v"*5 + "-"*10)
    print(f"=== Description")
    print(row["description"])
    print(f"=== Context")
    print(row["context"])


IndexError: single positional indexer is out-of-bounds

In [71]:
rows_to_drop = [ 2673, 1527, 2782, 149, 155, 164, 1564, 306, 430, 438, 466, 514, 545, 876, 877, 853, 565, 865, 1503, 867, 868, 801, 805, 742, 765, 832, 858, 990, 1059, 1110, 1124, 1168, 1197, 1205, 1211, 1244, 1248, 1272, 1290, 1545, 1495, 1531, 1553, 1561, 1562, 1689, 1740, 2214, 1810, 1847, 1865, 1868, 1950, 1974, 2359, 2012, 2080, 2613, 2345, 2364, 2457, 2476, 2609, 2556, 2634, 2800, 2827 ]
df_copy = dataset.copy()

In [73]:
df_no_duplicates = df_copy.drop(rows_to_drop)

In [79]:
to_save = df_no_duplicates[['query', 'description', 'context']]
to_save.reset_index(drop=True, inplace=True)
# "../datasets/final_queries_v1.8.json"

In [80]:
to_save.tail()

,query,description,context
2766,select ?feat ?en (count(*) as ?total)\nwith {\...,Grammatical features used on English nouns and...,
2767,SELECT ?country ?countryLabel ?lifeExpectancy\...,List of countries sorted by life expectancy,
2768,SELECT ?item ?itemLabel ?coord ?image\nWHERE\n...,Railway bridges in Germany and Austria,
2769,SELECT DISTINCT ?train_wreck ?train_wreckLabel...,Number of deaths in train wrecks per country,
2770,select distinct ?item1 ?article1 ?item2 ?artic...,Pairs of people who both have an English #Wiki...,


In [81]:
to_save.to_json("../datasets/final_queries_v1.8.json")

# Remove duplicates of final_queries 1.1

In [1]:
import sys
from pathlib import Path
sys.path.append(Path("../modules/").absolute().__str__())

from data_utils import load_dataset

In [2]:
dataset = load_dataset("../datasets/final_queries_v1.1.json")
dataset.head()

,query,description,context,prompt,num_tokens
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...,<s>[INST] <<SYS>>This is a conversation betwee...,267
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...,<s>[INST] <<SYS>>This is a conversation betwee...,296
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...,<s>[INST] <<SYS>>This is a conversation betwee...,197
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...,<s>[INST] <<SYS>>This is a conversation betwee...,203
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...,<s>[INST] <<SYS>>This is a conversation betwee...,186


In [3]:
df_drop = dataset.drop_duplicates(subset=["query"])[['query', 'description', 'context']]
df_drop.head()

,query,description,context
0,SELECT ?property ?propertyType ?propertyLabel ...,Wikidata properties in numerical order,Counting stuff on Wikidata\nAll Wikidata prope...
1,SELECT ?id ?idLabel ?idDescription ?new{\n?id ...,Wikidata properties excluding external IDs,Counting stuff on Wikidata\nVariation of the a...
2,SELECT (COUNT(?article) AS ?count)\nWHERE {\n?...,,Counting stuff on Wikidata\nCount of scientifi...
3,SELECT (COUNT(DISTINCT ?article) AS ?count)\nW...,Count of fictional characters,Counting stuff on Wikidata\nCount of fictional...
4,SELECT (COUNT(?item) AS ?count)\nWHERE { ?item...,Count of items with coordinate locations,Counting stuff on Wikidata\nCount of items wit...


In [4]:
to_save = df_drop.reset_index(drop=True, inplace=False)
to_save.tail()

,query,description,context
2774,select ?feat ?en (count(*) as ?total)\nwith {\...,Grammatical features used on English nouns and...,
2775,SELECT ?country ?countryLabel ?lifeExpectancy\...,List of countries sorted by life expectancy,
2776,SELECT ?item ?itemLabel ?coord ?image\nWHERE\n...,Railway bridges in Germany and Austria,
2777,SELECT DISTINCT ?train_wreck ?train_wreckLabel...,Number of deaths in train wrecks per country,
2778,select distinct ?item1 ?article1 ?item2 ?artic...,Pairs of people who both have an English #Wiki...,


In [5]:
to_save.to_json("../datasets/final_queries_v1.1_modified.json")